In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import numpy as np # advanced math library
import matplotlib.pyplot as plt # MATLAB like plotting routines
import random # for generating random numbers
import tensorflow as tf
from keras.datasets import mnist # MNIST dataset is included in Keras
from keras.models import Sequential # Model type to be used
from keras import models
from keras.layers.core import Dense, Dropout, Activation # Types of layers to be used in our model
from keras.utils import np_utils # NumPy related tools
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, Flatten
from keras.layers import BatchNormalization

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train = X_train.reshape(60000, 28, 28, 1) #add an additional dimension to represent the single-channel
X_test = X_test.reshape(10000, 28, 28, 1)
X_train = X_train.astype('float32') # change integers to 32-bit floating point numbers
X_test = X_test.astype('float32')
X_train /= 255 # normalize each value for each pixel for the entire vector for each input
X_test /= 255
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

In [ ]:
# one-hot format classes
nb_classes = 10 # number of unique digits
Training matrix shape (60000, 28, 28, 1)
Testing matrix shape (10000, 28, 28, 1)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
model = Sequential() # Linear stacking of layers
# Convolution Layer 1
model.add(Conv2D(32, (3, 3), input_shape=(28,28,1))) # 32 different 3x3 kernels -- so 32 feature maps
model.add(BatchNormalization(axis=-1)) # normalize each feature map before activation
convLayer01 = Activation('relu') # activation
model.add(convLayer01)
# Convolution Layer 2
model.add(Conv2D(32, (3, 3))) # 32 different 3x3 kernels -- so 32 feature maps
model.add(BatchNormalization(axis=-1)) # normalize each feature map before activation
model.add(Activation('relu')) # activation
convLayer02 = MaxPooling2D(pool_size=(2,2)) # Pool the max values over a 2x2 kernel
model.add(convLayer02)
# Convolution Layer 3
model.add(Conv2D(64,(3, 3))) # 64 different 3x3 kernels -- so 64 feature maps
model.add(BatchNormalization(axis=-1)) # normalize each feature map before activation
convLayer03 = Activation('relu') # activation
model.add(convLayer03)
# Convolution Layer 4
model.add(Conv2D(64, (3, 3))) # 64 different 3x3 kernels -- so 64 feature maps
model.add(BatchNormalization(axis=-1)) # normalize each feature map before activation
model.add(Activation('relu')) # activation
convLayer04 = MaxPooling2D(pool_size=(2,2)) # Pool the max values over a 2x2 kernel
model.add(convLayer04)
model.add(Flatten()) # Flatten final 4x4x64 output matrix into a 1024-length vector
# Fully Connected Layer 5
model.add(Dense(512)) # 512 FCN nodes
model.add(BatchNormalization()) # normalization
model.add(Activation('relu')) # activation
# Fully Connected Layer 6
model.add(Dropout(0.2)) # 20% dropout of randomly selected nodes
model.add(Dense(10)) # final 10 FCN nodes
model.add(Activation('softmax')) # softmax activation
model.summary()


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,height_shift_range=0.08, zoom_range=0.08)
test_gen = ImageDataGenerator()


In [ ]:
train_generator = gen.flow(X_train, Y_train, batch_size=128)
test_generator = test_gen.flow(X_test, Y_test, batch_size=128)

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=60000//128, epochs=15, verbose=1,
 validation_data=test_generator, validation_steps=10000//128)
model.save_weights('mnist_cnn_weights.h5')
model.load_weights('mnist_cnn_weights.h5')


## Generate a random image

In [ ]:
from keras.preprocessing import image
import numpy as np
from keras.utils import img_to_array, load_img, array_to_img

In [ ]:
img_path = 'image path' # path to your input image
img = load_img(img_path, target_size=(28,28), color_mode='grayscale')
img_tensor = img_to_array(img)
img_tensor /= 255.
img_tensor = np.expand_dims(img_tensor, axis=0)

In [ ]:
# Define a list of layer names to visualize
layer_names = []
for layer in model.layers[:15]:
 layer_names.append(layer.name)

In [ ]:
# Create a model that returns the activations for the named layers
layer_outputs = [layer.output for layer in model.layers]
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

In [ ]:
# Get the activations for the input image
activations = activation_model.predict(img_tensor)

In [ ]:
# Visualize the activations
import matplotlib.pyplot as plt
for layer_name, layer_activation in zip(layer_names, activations):
 n_features = layer_activation.shape[-1]
 size = layer_activation.shape[1]
 n_cols = n_features // 16
 if layer_activation.ndim == 2:
 display_grid = layer_activation
 elif layer_activation.ndim == 4:
 display_grid = np.zeros((size * n_cols, 16 * size))
 for col in range(n_cols):
 for row in range(16):
 channel_image = layer_activation[0, :, :, col * 16 + row]
 channel_image -= channel_image.mean()
 channel_image /= channel_image.std()
 channel_image *= 64
 channel_image += 128
 channel_image = np.clip(channel_image, 0, 255).astype('uint8')
 display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image
 scale = 1. / size
 plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
 plt.title(layer_name)
 plt.grid(False)
 plt.imshow(display_grid, aspect='auto', cmap='viridis')
 #plt.savefig(f"{layer_name}.png")
plt.show()



